<a href="https://colab.research.google.com/github/kairamilanifitria/PurpleBox-Intern/blob/main/03_06_Retrieval_LLM_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# trial : separate data tables

In [ ]:
!pip install supabase numpy psycopg2

In [ ]:
import os
import json
import numpy as np
from supabase import create_client, Client

# Initialize Supabase
SUPABASE_URL = ""
SUPABASE_KEY = ""

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)


In [ ]:
import json
import torch
import uuid  # Import the missing module
from transformers import AutoTokenizer, AutoModel
from supabase import create_client

# Load Embedding Model
tokenizer = AutoTokenizer.from_pretrained("Alibaba-NLP/gte-multilingual-base")
model = AutoModel.from_pretrained("Alibaba-NLP/gte-multilingual-base").to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().tolist()

# Load JSON chunks
json_file_path = "/content/17.json"
with open(json_file_path, "r", encoding="utf-8") as json_file:
    json_chunks = json.load(json_file)

def store_chunks_in_supabase(chunks):
    """Stores chunks into Supabase, differentiating between text and tables."""
    document_entries = []
    table_entries = []

    for chunk in chunks:
        chunk_id = str(uuid.uuid4())  # Generate unique chunk_id

        if "content" in chunk and chunk["content"]:
            content = chunk["content"]
            embedding = get_embedding(content)

            document_entries.append({
                "chunk_id": chunk_id,
                "content": content,
                "embedding": embedding,
                "metadata": chunk["metadata"],
                "type": "text"
            })

        if "table" in chunk and chunk["table"]:
            table_data = chunk["table"]
            table_entries.append({
                "chunk_id": chunk_id,
                "table_data": json.dumps(table_data, ensure_ascii=False),
                "metadata": chunk["metadata"]
            })

    # Batch insert into Supabase for efficiency
    if document_entries:
        supabase.table("documents").insert(document_entries).execute()
    if table_entries:
        supabase.table("tables").insert(table_entries).execute()

# Store chunks in Supabase
store_chunks_in_supabase(json_chunks)

print("Chunks with embeddings stored successfully in Supabase!")

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

The repository for Alibaba-NLP/gte-multilingual-base contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Alibaba-NLP/gte-multilingual-base.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Chunks with embeddings stored successfully in Supabase!


In [ ]:
import ast
import numpy as np
import re
from scipy.spatial.distance import cosine

def query_supabase(user_query):
    """Retrieves both text and table chunks based on query."""

    #### 🔹 Step 1: Retrieve Text Chunks (Vector Search) ####
    query_embedding = np.array(get_embedding(user_query), dtype=np.float32).flatten()
    print(f"Query embedding shape: {query_embedding.shape}")  # Debugging

    response_text = supabase.table("documents").select("chunk_id, content, embedding").execute()
    text_results = []

    for record in response_text.data:
        chunk_embedding = record["embedding"]

        # Convert stored string embeddings to list if needed
        if isinstance(chunk_embedding, str):
            chunk_embedding = ast.literal_eval(chunk_embedding)

        chunk_embedding = np.array(chunk_embedding, dtype=np.float32).flatten()
        print(f"Chunk {record['chunk_id']} embedding shape: {chunk_embedding.shape}")  # Debugging

        if chunk_embedding.shape == query_embedding.shape:
            similarity = 1 - cosine(query_embedding, chunk_embedding)
            text_results.append((record["chunk_id"], "text", record["content"], similarity))
        else:
            print(f"⚠️ Skipping chunk {record['chunk_id']} due to shape mismatch.")

    #### 🔹 Step 2: Retrieve Table Chunks (Improved Keyword Search) ####
    response_tables = supabase.table("tables").select("chunk_id, table_data").execute()
    table_results = []

    query_words = set(re.findall(r'\w+', user_query.lower()))  # Extract words from query

    for record in response_tables.data:
        table_data = record["table_data"].lower()
        table_words = set(re.findall(r'\w+', table_data))  # Extract words from table

        common_words = query_words.intersection(table_words)  # Count overlapping words
        match_score = len(common_words) / max(len(query_words), 1)  # Normalize score

        if match_score > 0:  # Only include tables with at least one match
            table_results.append((record["chunk_id"], "table", table_data, match_score))

    #### 🔹 Step 3: Merge & Sort Results ####
    all_results = text_results + table_results
    all_results.sort(key=lambda x: x[3], reverse=True)  # Sort by relevance

    return all_results[:5]  # Return top 5 results


In [ ]:
# Example usage
user_query = "Background of reports"
retrieved_chunks = query_supabase(user_query)

# Display results
for chunk in retrieved_chunks:
    print(f"Chunk ID: {chunk[0]}\nType: {chunk[1]}\nContent: {chunk[2][:300]}...\nRelevance: {chunk[3]:.4f}\n")

Query embedding shape: (768,)
Chunk b5ea80d5-dc09-4096-9603-833ab73a3504 embedding shape: (768,)
Chunk d73ba009-0f60-47de-a372-8fc11c669432 embedding shape: (768,)
Chunk 56322f14-18e5-41c8-9a94-69cc2dbc258b embedding shape: (768,)
Chunk 72e54398-d8bb-4ced-87c3-db5ba6e4e1ee embedding shape: (768,)
Chunk aca32bb6-707e-44ba-a6cc-3d3f80b847fc embedding shape: (768,)
Chunk d8a92179-46f3-44f4-a51b-26243f89c2b2 embedding shape: (768,)
Chunk 5531a33f-a880-42db-b934-a66d255cfcc8 embedding shape: (768,)
Chunk 4711385f-c69a-4ebf-ba9b-f1873ff82f60 embedding shape: (768,)
Chunk 1256b5a0-4ef8-458b-affe-b383a37b841f embedding shape: (768,)
Chunk 71c30bd2-e19f-4ae7-8ac7-e5e6ed238067 embedding shape: (768,)
Chunk 3c7266c0-b157-4107-9f05-a97d3d2eef84 embedding shape: (768,)
Chunk 19be3d97-ba94-4fc1-ad18-defb36c4e320 embedding shape: (768,)
Chunk 342f37ff-29b3-433f-89cc-a8ee114dfaf0 embedding shape: (768,)
Chunk 846094a2-0c29-4ab0-8963-20feb220b1e2 embedding shape: (768,)
Chunk ID: 71c30bd2-e19f-4ae7-8ac

In [ ]:
# Example usage
user_query = "Number of participants in Ireland"
retrieved_chunks = query_supabase(user_query)

# Display results
for chunk in retrieved_chunks:
    print(f"Chunk ID: {chunk[0]}\nType: {chunk[1]}\nContent: {chunk[2][:300]}...\nRelevance: {chunk[3]:.4f}\n")

Query embedding shape: (768,)
Chunk b5ea80d5-dc09-4096-9603-833ab73a3504 embedding shape: (768,)
Chunk d73ba009-0f60-47de-a372-8fc11c669432 embedding shape: (768,)
Chunk 56322f14-18e5-41c8-9a94-69cc2dbc258b embedding shape: (768,)
Chunk 72e54398-d8bb-4ced-87c3-db5ba6e4e1ee embedding shape: (768,)
Chunk aca32bb6-707e-44ba-a6cc-3d3f80b847fc embedding shape: (768,)
Chunk d8a92179-46f3-44f4-a51b-26243f89c2b2 embedding shape: (768,)
Chunk 5531a33f-a880-42db-b934-a66d255cfcc8 embedding shape: (768,)
Chunk 4711385f-c69a-4ebf-ba9b-f1873ff82f60 embedding shape: (768,)
Chunk 1256b5a0-4ef8-458b-affe-b383a37b841f embedding shape: (768,)
Chunk 71c30bd2-e19f-4ae7-8ac7-e5e6ed238067 embedding shape: (768,)
Chunk 3c7266c0-b157-4107-9f05-a97d3d2eef84 embedding shape: (768,)
Chunk 19be3d97-ba94-4fc1-ad18-defb36c4e320 embedding shape: (768,)
Chunk 342f37ff-29b3-433f-89cc-a8ee114dfaf0 embedding shape: (768,)
Chunk 846094a2-0c29-4ab0-8963-20feb220b1e2 embedding shape: (768,)
Chunk ID: f2061f00-3b95-46e5-907

# Chat with LLM

In [ ]:
!pip install groq

In [ ]:
import ast
import numpy as np
import re
import requests
from scipy.spatial.distance import cosine

# Your Groq API key
GROQ_API_KEY = ""

def query_supabase(user_query):
    """Retrieves both text and table chunks based on query."""

    #### 🔹 Step 1: Retrieve Text Chunks (Vector Search) ####
    query_embedding = np.array(get_embedding(user_query), dtype=np.float32).flatten()
    response_text = supabase.table("documents").select("chunk_id, content, embedding").execute()
    text_results = []

    for record in response_text.data:
        chunk_embedding = record["embedding"]

        # Convert stored string embeddings to list if needed
        if isinstance(chunk_embedding, str):
            chunk_embedding = ast.literal_eval(chunk_embedding)

        chunk_embedding = np.array(chunk_embedding, dtype=np.float32).flatten()

        if chunk_embedding.shape == query_embedding.shape:
            similarity = 1 - cosine(query_embedding, chunk_embedding)
            text_results.append((record["chunk_id"], "text", record["content"], similarity))

    #### 🔹 Step 2: Retrieve Table Chunks (Improved Keyword Search) ####
    response_tables = supabase.table("tables").select("chunk_id, table_data").execute()
    table_results = []

    query_words = set(re.findall(r'\w+', user_query.lower()))  # Extract words from query

    for record in response_tables.data:
        table_data = record["table_data"].lower()
        table_words = set(re.findall(r'\w+', table_data))  # Extract words from table

        common_words = query_words.intersection(table_words)  # Count overlapping words
        match_score = len(common_words) / max(len(query_words), 1)  # Normalize score

        if match_score > 0:  # Only include tables with at least one match
            table_results.append((record["chunk_id"], "table", table_data, match_score))

    #### 🔹 Step 3: Merge & Sort Results ####
    all_results = text_results + table_results
    all_results.sort(key=lambda x: x[3], reverse=True)  # Sort by relevance

    return all_results[:5]  # Return top 5 results


def call_groq_llm(user_query, retrieved_chunks):
    """Send the query along with retrieved context to Groq API and return the response."""

    # Print retrieved chunks for debugging
    print("\n🔹 Retrieved Chunks:")
    for i, chunk in enumerate(retrieved_chunks, 1):
        print(f"Chunk {i} (ID: {chunk[0]}, Type: {chunk[1]}):\n{chunk[2][:500]}...\nRelevance: {chunk[3]:.4f}\n")

    # Prepare context for LLM
    context_text = "\n\n".join([f"Chunk {i+1}: {chunk[2]}" for i, chunk in enumerate(retrieved_chunks)])

    prompt = f"""You are an intelligent assistant. Use the following retrieved information to answer the user's query.

    Context:
    {context_text}

    User's Question: {user_query}

    Provide a clear and concise response.
    """

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "qwen-qwq-32b",  # Adjust this based on your Groq model selection
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        answer = response.json()["choices"][0]["message"]["content"]
        print("\n🔹 Chatbot Response:\n", answer)
        return answer
    else:
        print("\n⚠️ Error:", response.text)
        return None



In [ ]:
# Example usage
user_query = "Number of participants in Ireland"
retrieved_chunks = query_supabase(user_query)

if retrieved_chunks:
    call_groq_llm(user_query, retrieved_chunks)
else:
    print("No relevant information found.")



🔹 Retrieved Chunks:
Chunk 1 (ID: f2061f00-3b95-46e5-907f-05b74dd09987, Type: table):
{"headers": ["country", "number of participants"], "rows": [["botswana", "2"], ["ethiopia", "1"], ["ireland", "1"], ["kenya", "1"], ["lesotho", "4"], ["malawi", "3"], ["nigeria", "2"], ["south africa", "12"], ["sweden", "1"], ["uganda", "1"], ["united kingdom", "2"], ["united states (involved with lesotho  programme)", "2"], ["zimbabwe", "3"], ["total", "35"]]}...
Relevance: 0.8000

Chunk 2 (ID: aca32bb6-707e-44ba-a6cc-3d3f80b847fc, Type: text):
## Participants and process  
Thirty-five people  participated  in  a  2-h  workshop  and included trainers and trainees from nine African countries, the United Kingdom, United States and Sweden (see Table 1). South  Africa  was  represented  by  the  universities  of  Cape Town,  Limpopo,  Pretoria,  Sefako  Makgatho,  Stellenbosch, Walter Sisulu and Witwatersrand.  
We started with an introduction and then divided into buzz pairs (pairs were allowed to form 

In [ ]:
# Example usage
user_query = "Number of participants in Lesotho"
retrieved_chunks = query_supabase(user_query)

if retrieved_chunks:
    call_groq_llm(user_query, retrieved_chunks)
else:
    print("No relevant information found.")



🔹 Retrieved Chunks:
Chunk 1 (ID: aca32bb6-707e-44ba-a6cc-3d3f80b847fc, Type: text):
## Participants and process  
Thirty-five people  participated  in  a  2-h  workshop  and included trainers and trainees from nine African countries, the United Kingdom, United States and Sweden (see Table 1). South  Africa  was  represented  by  the  universities  of  Cape Town,  Limpopo,  Pretoria,  Sefako  Makgatho,  Stellenbosch, Walter Sisulu and Witwatersrand.  
We started with an introduction and then divided into buzz pairs (pairs were allowed to form spontaneously, regardless of the tra...
Relevance: 0.8174

Chunk 2 (ID: 19be3d97-ba94-4fc1-ad18-defb36c4e320, Type: text):
## Reflection on workshop  
The group appreciated the richness of the discussion and the value  of  having  a  variety  of  countries  represented  in  the workshop. The group members expressed feeling encouraged and felt motivated to use 'small moments, little bits, part of the mini-CEX' during learning interactions in the wo